In [50]:
input_query = 'ReAct framework for agents'

In [51]:
import psycopg2
import os

def connection():
    """Creates and returns a new database connection."""
    try:
        conn = psycopg2.connect(
            user=os.environ["MY_INTEGRATION_USER"],
            password=os.environ["MY_INTEGRATION_PASSWORD"],
            host=os.environ["MY_INTEGRATION_HOST"],
            port=os.environ["MY_INTEGRATION_PORT"],
            database=os.environ["MY_INTEGRATION_DATABASE"]
        )
        
        # Test the connection
        with conn.cursor() as cursor:
            cursor.execute("SELECT version();")
            record = cursor.fetchone()
        
        return conn  # Return the connection object if successful

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to database", error)
        return None  # Return None if connection was not successful

conn = connection()

In [80]:
from rich.console import Console
from rich.table import Table
from rich.text import Text

def display_query_papers(job_query):
    # Create a console object for Rich output
    console = Console()

    # Connect to SQLite database
    conn = connection()
    c = conn.cursor()

    # Fetch the number of ranks already printed for the given query
    c.execute("SELECT printed_ranks FROM jobs WHERE query = %s", (job_query,))
    result = c.fetchone()
    printed_ranks = result[0] if result else 0  # Use the fetched number or default to 0 if not found

    start_rank = printed_ranks + 1  # Start from the next rank

    # Fetch records for the given query starting from the next rank to be printed
    c.execute("""
        SELECT * FROM Query_Papers 
        WHERE query = %s AND final_rank >= %s AND final_rank IS NOT NULL 
        AND relevant_answer IS NOT NULL AND paper_stats IS NOT NULL 
        AND paper_metadata_filtered IS NOT NULL AND download_link IS NOT NULL
        ORDER BY final_rank ASC
        LIMIT 10
    """, (job_query, start_rank))

    # Fetch the column names
    columns = [description[0] for description in c.description]

    rows = c.fetchall()

    # Counter for the number of ranks printed during this function call
    ranks_printed_now = 0

    if rows:
        # Initialize a Rich table with improved formatting
        table = Table(show_header=True, title=job_query, expand=True, leading=1, show_lines=True)
        table.add_column("No.", style="cyan", justify="right", ratio=1)
        table.add_column("Paper", overflow="fold", ratio=20)  # This has twice the ratio of "Details", meaning it will be larger
        table.add_column("Details", overflow="fold", ratio=8)  # Half the 'ratio' of "Paper", making it relatively smaller
        table.add_column("Link", justify="center", ratio=2)
        for row in rows:
            # Extract the necessary fields from the row
            final_rank = row[columns.index('final_rank')]
            arxiv_link = row[columns.index('arxiv_link')]
            relevant_answer = row[columns.index('relevant_answer')]
            paper_stats = json.loads(row[columns.index('paper_stats')])
            paper_metadata_filtered = json.loads(row[columns.index('paper_metadata_filtered')])
            
            # Format extracted data
            title = paper_metadata_filtered.get('title', 'N/A')
            abstract = paper_metadata_filtered.get('abstract', 'N/A')
            abstract = (abstract[:197] + '...') if len(abstract) > 200 else abstract
            published_date = paper_metadata_filtered.get('published_date', 'N/A').split('T')[0] if paper_metadata_filtered.get('published_date', 'N/A') != 'N/A' else 'N/A'
            authors = paper_metadata_filtered.get('authors', ['N/A'])
            authors_str = ", ".join(authors[:3]) + ("..." if len(authors) > 3 else "")
            citations = paper_stats.get('citations', 'N/A')
            versions = paper_stats.get('versions', 'N/A')

            # Add the clickable 'Link' text
            link_text = f"[link={arxiv_link}]Link[/link]"

            # Format the Paper and Details columns
            paper_column = Text(f"{title}\n\nLLM response: {relevant_answer}\n\nAbstract: {abstract}", justify="left")
            details_column = Text(f"Citations: {citations}\nVersions: {versions}\nDate Published: {published_date}\nAuthors: {authors_str} \n", justify="left")
            
            # Add row with formatted data
            table.add_row(str(final_rank), paper_column, details_column, link_text)
            table.add_section()
            table.add_row()

            ranks_printed_now += 1

        # Update the number of printed ranks in the jobs table for this query
        new_total_printed = printed_ranks + ranks_printed_now
        c.execute("UPDATE jobs SET printed_ranks = %s WHERE query = %s", (new_total_printed, job_query))
        conn.commit()
    # Print the table to the console
    if ranks_printed_now > 0:
        console.print(table)
    # Closing database connections
    c.close()
    conn.close()

    return ranks_printed_now  # Optionally return the number of ranks printed in this call

# # Example usage
# job_query = "Top academic papers on ReAct framework for agents"
# ranks_printed_now = display_query_papers(job_query)
# print(f"Ranks printed this time: {ranks_printed_now}")


In [56]:
def add_new_job(query):
    # Connect to the SQLite database
    conn = connection()
    c = conn.cursor()

    # SQL statement to insert a new job
    c.execute("INSERT INTO jobs (query, job_status) VALUES (%s, 'new')", (query,))
    print("\nHello there, we're busy working on your query: '{}'.".format(job_query))
    # Commit the changes and close the connection
    conn.commit()
    conn.close()

# Usage
job_query = "Top academic papers on " + input_query
add_new_job(job_query)
print(job_query)

Top academic papers on ReAct framework for agents


In [81]:
import sqlite3
import time
import datetime  # Import the datetime module
import os  # Import the os module for clearing the terminal
from IPython.display import clear_output

def wait_for_job_completion(job_query):
    # Connect to the SQLite database
    conn = connection()
    c = conn.cursor()
    counter = 0  # Initialize the counter
    try:
        while True:  # Keep checking until the job is done
            # SQL statement to find the status of a job given its query
            c.execute("SELECT job_status FROM jobs WHERE query = %s", (job_query,))
            result = c.fetchone()
            if result:
                job_status = result[0]
                if job_status == 'done':
                    print("\nThe job '{}' is complete.".format(job_query))  # Ensure new line before final status
                    display_query_papers(job_query)
                    break  # Exit the loop if the job is done
                elif job_status == 'running':
                    print(f".", end=' ')  # Print counter with "Running..."
                    # clear_output(wait=True)  # Clear output and wait for the next
                    counter += 1
                    # print(f"{counter}", end=' ')  # Print counter with "Running..."
                    # print("\nThe status of the job '{}' is currently '{}'. Waiting for completion...".format(job_query, job_status))
                    display_query_papers(job_query)
                    time.sleep(1)
                # else:
                #     print("\nThe status of the job '{}' is currently '{}'. Waiting for completion...".format(job_query, job_status))
                #     display_query_papers(job_query)  # Indented to match the 'else' block
            else:
                print("\nHello there, we're busy working on your query: '{}'.".format(job_query))
                break  # Exit the loop if no such job exists
    except Exception as e:
        print(f"An error occurred: {e}")

# Usage
wait_for_job_completion(job_query)



The job 'Top academic papers on ReAct framework for agents' is complete.


                                 Top academic papers on ReAct framework for agents                                 
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ N… ┃ Paper                                                                 ┃ Details                    ┃ Link  ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│  1 │ Chain-of-Thought Prompting Elicits Reasoning in Large Language Models │ Citations: 3769            │ ]8;id=885907;https://arxiv.org/abs/2201.11903\Link]8;;\  │
│    │                                                                       │ Versions: 12               │       │
│    │ LLM response: Paper: "Chain-of-Thought Prompting Elicits Reasoning in │ Date Published: 2022-01-28 │       │
│    │ Large Language Models" discusses enhancing reasoning in models.       │ Authors: Jason Wei, Xuezhi │       │
│    │                                                                       │ Wang, Dale Schuurmans...   │       │
│    │ Abstract: We explore how generating a chain of thought -- a series of │                            │       │
│    │ intermediate                                                          │                            │       │
│    │ reasoning steps -- significantly improves the ability of large        │                            │       │
│    │ language models                                                       │                            │       │
│    │ to perform complex reasoning. In particular, ...                      │                            │       │
│    │                                                                       │                            │       │
│    │                                                                       │                            │       │
│    │                                                                       │                            │       │
│  2 │ FEVER: a large-scale dataset for Fact Extraction and VERification     │ Citations: 1288            │ ]8;id=167110;https://arxiv.org/abs/1803.05355\Link]8;;\  │
│    │                                                                       │ Versions: 14               │       │
│    │ LLM response: FEVER dataset introduces claim verification challenges, │ Date Published: 2018-03-14 │       │
│    │ aiding progress in textual source verification.                       │ Authors: James Thorne,     │       │
│    │                                                                       │ Andreas Vlachos, Christos  │       │
│    │ Abstract: In this paper we introduce a new publicly available dataset │ Christodoulopoulos...      │       │
│    │ for verification                                                      │                            │       │
│    │ against textual sources, FEVER: Fact Extraction and VERification. It  │                            │       │
│    │ consists                                                              │                            │       │
│    │ of 185,445 claims generated by altering se...                         │                            │       │
│    │                                                                       │                            │       │
│    │                                                                       │                            │       │
│    │                                                                       │                            │       │
│  3 │ HotpotQA: A Dataset for Diverse, Explainable Multi-hop Question       │ Citations: 1613            │ ]8;id=98795;https://arxiv.org/abs/1809.09600\Link]8;;\  │
│    │   Answering                                                           │ Versions: 9                │       │
│    │                                                                       │ Date Published: 2018-09-25 │       │
│    │ L

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d52007a-f237-4857-b1f1-3ccb95216ee4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>